In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import LabelEncoder

In [61]:
data = pd.read_csv("https://www.ee.iitb.ac.in/~asethi/Dump/MouseTrain.csv")
data.head()

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,...,BCL2_N,pS6_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment_Behavior
0,0.503644,0.747193,0.430175,2.816329,5.990152,0.218830,0.177565,2.373744,0.232224,1.750936,...,NaN,0.106305,0.108336,0.427099,0.114783,0.131790,0.128186,1.675652,Control,Memantine_C/S
1,0.514617,0.689064,0.411770,2.789514,5.685038,0.211636,0.172817,2.292150,0.226972,1.596377,...,NaN,0.106592,0.104315,0.441581,0.111974,0.135103,0.131119,1.743610,Control,Memantine_C/S
2,0.509183,0.730247,0.418309,2.687201,5.622059,0.209011,0.175722,2.283337,0.230247,1.561316,...,NaN,0.108303,0.106219,0.435777,0.111883,0.133362,0.127431,1.926427,Control,Memantine_C/S
3,0.442107,0.617076,0.358626,2.466947,4.979503,0.222886,0.176463,2.152301,0.207004,1.595086,...,NaN,0.103184,0.111262,0.391691,0.130405,0.147444,0.146901,1.700563,Control,Memantine_C/S
4,0.434940,0.617430,0.358802,2.365785,4.718679,0.213106,0.173627,2.134014,0.192158,1.504230,...,NaN,0.104784,0.110694,0.434154,0.118481,0.140314,0.148380,1.839730,Control,Memantine_C/S


In [62]:
# # convert string labels to integer labels
# data['Genotype'] = data['Genotype'].map({'Control': -1, 'Ts65Dn': 1})
# data['Treatment_Behavior'] = data['Treatment_Behavior'].map({'Memantine_C/S': 0, 'Memantine_C/S': 1, 'Saline_C/S': 2, 'Saline_S/': 3})

# split data into X (features) and y (labels)
X = data.iloc[:, 0:77]  # select columns 1-77 as features
y1 = data['Genotype']
y2 = data['Treatment_Behavior']
y1

0      Control
1      Control
2      Control
3      Control
4      Control
        ...   
757     Ts65Dn
758     Ts65Dn
759     Ts65Dn
760     Ts65Dn
761     Ts65Dn
Name: Genotype, Length: 762, dtype: object

In [63]:
## Q.2 (a)
# drop columns with variance greater than 0.01
X = X.loc[:, X.var() < 0.01]

# print variance of each feature
print(X.var())
X.shape

BDNF_N             0.002166
pAKT_N             0.001562
pBRAF_N            0.000705
pCREB_N            0.000965
pJNK_N             0.002615
PKCA_N             0.002366
pMEK_N             0.001932
pRSK_N             0.004140
CAMKII_N           0.002678
CREB_N             0.000610
JNK_N              0.001136
MEK_N              0.001496
RSK_N              0.000782
APP_N              0.003283
MTOR_N             0.004105
P38_N              0.008802
AMPKA_N            0.003595
NR2B_N             0.008086
pNUMB_N            0.003788
RAPTOR_N           0.002631
TIAM1_N            0.003821
NUMB_N             0.000884
pGSK3B_N           0.000436
CDK5_N             0.001097
RRP1_N             0.001111
BAX_N              0.000357
ARC_N              0.000223
ERBB4_N            0.000251
nNOS_N             0.000639
Tau_N              0.004927
GFAP_N             0.000195
GluR3_N            0.001317
GluR4_N            0.000609
IL1B_N             0.007322
P3525_N            0.000914
SNCA_N             0

(762, 46)

In [64]:
## Q.2 (b) removing one of the columns whose correlation value exceeds 0.8
# calculate absolute correlation matrix
corr_matrix = X.corr().abs()

# set upper triangle values to zero to only consider unique pairs of features
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# set threshold for correlation
alpha = 0.75

# create list of columns to drop based on correlation threshold
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > alpha)]

# drop columns from X
X = X.drop(columns=to_drop)
X.isnull().sum()

C:\Users\sept2\AppData\Local\Temp\ipykernel_28796\1466605885.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


BDNF_N               0
pAKT_N               0
pCREB_N              0
PKCA_N               0
pRSK_N               0
APP_N                0
MTOR_N               0
AMPKA_N              0
pNUMB_N              0
NUMB_N               0
pGSK3B_N             0
CDK5_N               0
RRP1_N               0
BAX_N                0
ARC_N                0
ERBB4_N              0
nNOS_N               0
Tau_N                0
GFAP_N               0
GluR3_N              0
GluR4_N              0
IL1B_N               0
P3525_N              0
SNCA_N               0
pGSK3B_Tyr216_N      0
SHH_N                0
BAD_N              180
pCFOS_N             60
SYP_N                0
H3AcK18_N          150
dtype: int64

In [65]:
X.shape

(762, 30)

In [66]:
##Q.2 (c)




In [67]:
##Q.3
# create an iterative imputer object
imp = IterativeImputer(max_iter=10, random_state=0)

# fit the imputer to the dataframe
imp.fit(X)

# impute missing values in the test dataframe
X = pd.DataFrame(imp.transform(X), columns=X.columns)

# print the imputed dataframe
X.shape
X.isnull().sum()

BDNF_N             0
pAKT_N             0
pCREB_N            0
PKCA_N             0
pRSK_N             0
APP_N              0
MTOR_N             0
AMPKA_N            0
pNUMB_N            0
NUMB_N             0
pGSK3B_N           0
CDK5_N             0
RRP1_N             0
BAX_N              0
ARC_N              0
ERBB4_N            0
nNOS_N             0
Tau_N              0
GFAP_N             0
GluR3_N            0
GluR4_N            0
IL1B_N             0
P3525_N            0
SNCA_N             0
pGSK3B_Tyr216_N    0
SHH_N              0
BAD_N              0
pCFOS_N            0
SYP_N              0
H3AcK18_N          0
dtype: int64

In [68]:
##Q.4
scoring = ['accuracy', 'balanced_accuracy', 'f1_micro', 'f1_macro', 'f1_weighted', 'roc_auc_ovr']

In [69]:
##Q.5 (a) Linear SVM
svm_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('svc', LinearSVC())
])
svm_params = {'svc__C': np.logspace(-4, 4, 9)}

svm_gs = GridSearchCV(svm_pipeline, svm_params, cv=5, n_jobs=-1, scoring='balanced_accuracy')
svm_gs.fit(X, y1)
print("Best Linear SVM Parameters: ", svm_gs.best_params_)
svm_model = svm_gs.best_estimator_

Best Linear SVM Parameters:  {'svc__C': 0.01}


In [70]:
##Q.5 (b) RBF kernel SVM with kernel width and regularization as hyperparameters
rbf_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('svc', SVC(kernel='rbf'))
])
rbf_params = {'svc__C': np.logspace(-4, 4, 9), 'svc__gamma': np.logspace(-4, 4, 9)}

rbf_gs = GridSearchCV(rbf_pipeline, rbf_params, cv=5, n_jobs=-1, scoring='balanced_accuracy')
rbf_gs.fit(X, y1)
print("Best RBF SVM Parameters: ", rbf_gs.best_params_)
rbf_model = rbf_gs.best_estimator_

Best RBF SVM Parameters:  {'svc__C': 100.0, 'svc__gamma': 0.0001}


In [71]:
##Q.5 (c) neural network with single ReLU hidden layer and Softmax output (hyperparameters: number of neurons, weight decay)
nn_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('nn', MLPClassifier(max_iter=1000))
])
nn_params = {'nn__hidden_layer_sizes': [(32), (64), (128), (256)], 'nn__alpha': np.logspace(-4, 4, 9)}

nn_gs = GridSearchCV(nn_pipeline, nn_params, cv=5, n_jobs=-1, scoring='balanced_accuracy')
nn_gs.fit(X, y1)
print("Best Neural Network Parameters: ", nn_gs.best_params_)
nn_model = nn_gs.best_estimator_

Best Neural Network Parameters:  {'nn__alpha': 1.0, 'nn__hidden_layer_sizes': 32}


In [72]:
rf_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('rf', RandomForestClassifier())
])

rf_params = {'rf__max_depth': [2, 4, 6, 8], 'rf__max_features': ['sqrt', 'log2', None]}

rf_gs = GridSearchCV(rf_pipeline, rf_params, cv=5, n_jobs=-1, scoring='balanced_accuracy')
rf_gs.fit(X, y1)
print("Best Random Forest Parameters: ", rf_gs.best_params_)
rf_model = rf_gs.best_estimator_

Best Random Forest Parameters:  {'rf__max_depth': 4, 'rf__max_features': None}


In [73]:
# Get feature importances for each model
svm_importances = abs(svm_model.named_steps['svc'].coef_[0])
rbf_importances = None
nn_importances = np.mean(np.abs(nn_model.named_steps['nn'].coefs_[0]), axis=1)
rf_importances = rf_model.named_steps['rf'].feature_importances_

# Sort proteins by feature importance
protein_importances = pd.DataFrame({
    'Protein': X.columns,
    'SVM': svm_importances,
    'RBF-SVM': rbf_importances,
    'NN': nn_importances,
    'RF': rf_importances
})

protein_importances = protein_importances.sort_values('SVM', ascending=False)
protein_importances = protein_importances.set_index('Protein')
protein_importances = protein_importances.apply(lambda x: x/x.sum(), axis=0)
protein_importances = protein_importances.sort_values(by='RF', ascending=False)

print(protein_importances)

                      SVM RBF-SVM        NN        RF
Protein                                              
APP_N            0.202429     NaN  0.086301  0.472828
AMPKA_N          0.061049     NaN  0.045223  0.135624
GluR3_N          0.075802     NaN  0.043038  0.083462
MTOR_N           0.046886     NaN  0.038787  0.061637
Tau_N            0.021769     NaN  0.031074  0.035684
pNUMB_N          0.038175     NaN  0.036365  0.034321
BDNF_N           0.059219     NaN  0.031296  0.025751
pAKT_N           0.060161     NaN  0.039852  0.015771
pGSK3B_N         0.013242     NaN  0.019935  0.014148
BAX_N            0.025411     NaN  0.029101  0.012670
P3525_N          0.017617     NaN  0.039131  0.012591
GFAP_N           0.011721     NaN  0.020161  0.012462
ERBB4_N          0.044028     NaN  0.035726  0.010488
NUMB_N           0.022713     NaN  0.030048  0.008313
H3AcK18_N        0.019122     NaN  0.029092  0.007995
nNOS_N           0.033009     NaN  0.039462  0.006131
PKCA_N           0.013138   

In [74]:
test_df = pd.read_csv('https://www.ee.iitb.ac.in/~asethi/Dump/MouseTest.csv')
test_df.head()

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,...,BCL2_N,pS6_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment_Behavior
0,0.627582,0.954187,0.446958,2.930717,5.915039,0.197897,0.185599,1.765115,0.232257,1.808111,...,0.120835,0.148773,0.120885,0.541241,0.121674,0.158103,NaN,1.649983,Control,Memantine_C/S
1,0.651253,0.962021,0.464964,2.992689,5.975028,0.202810,0.188473,1.807824,0.259020,1.960691,...,0.130462,0.147660,0.128960,0.525966,0.126780,0.163405,NaN,1.638988,Control,Memantine_C/S
2,0.644346,0.967179,0.470056,3.073847,5.927379,0.205131,0.182590,1.783552,0.258795,2.147883,...,0.118851,0.143169,0.129154,0.508214,0.122796,0.160293,NaN,1.564925,Control,Memantine_C/S
3,0.568229,0.812018,0.393465,2.606780,5.808102,0.218211,0.179905,2.312324,0.209154,1.733692,...,0.116218,0.140063,0.127661,0.561721,0.123122,0.156108,NaN,1.682222,Control,Memantine_C/S
4,0.587038,0.863728,0.411203,2.757975,6.006657,0.223688,0.189400,2.375283,0.218538,1.843255,...,0.142739,0.140775,0.121389,0.534845,0.133996,0.180373,NaN,1.660352,Control,Memantine_C/S


In [75]:
# Get the column names of the selected features
selected_features = list(X.columns[:])

# Testing the models
test_features = test_df.iloc[:, 1:77].loc[:, selected_features]

# create an iterative imputer object
imp = IterativeImputer(max_iter=10, random_state=0)

# fit the imputer to the dataframe
imp.fit(test_features)

# impute missing values in the test dataframe
test_features = pd.DataFrame(imp.transform(test_features), columns=test_features.columns)

# print(test_features)

svm_preds = svm_model.predict(test_features)
rbf_preds = rbf_model.predict(test_features)
nn_preds = nn_model.predict(test_features)
rf_preds = rf_model.predict(test_features)

# Calculating balanced accuracy scores
svm_score = balanced_accuracy_score(test_df['Genotype'], svm_preds)
rbf_score = balanced_accuracy_score(test_df['Genotype'], rbf_preds)
nn_score = balanced_accuracy_score(test_df['Genotype'], nn_preds)
rf_score = balanced_accuracy_score(test_df['Genotype'], rf_preds)

# Printing the scores
print("Linear SVM Balanced Accuracy: ", svm_score)
print("RBF Kernel SVM Balanced Accuracy: ", rbf_score)
print("Neural Network Balanced Accuracy: ", nn_score)
print("Random Forest Balanced Accuracy: ", rf_score)

Linear SVM Balanced Accuracy:  0.8629629629629629
RBF Kernel SVM Balanced Accuracy:  0.8851851851851852
Neural Network Balanced Accuracy:  0.8685185185185185
Random Forest Balanced Accuracy:  0.825925925925926


In [ ]:
# Ref :
# https://scikit-learn.org/stable/modules/impute.html
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html
# https://chat.openai.com/chat - Have extensively used this to learn how to exactly implement, many places I have directly taken the code from there
# 19D110004 - I have also taken a lot of help from this friend and used his idea of implementation.
    